<a href="https://colab.research.google.com/github/ccarpenterg/introNLP/blob/master/04a_NLP_and_sequence_to_sequence_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP and Sequence-to-Sequence RNNs

In [1]:
!wget https://www.manythings.org/anki/spa-eng.zip
!unzip spa-eng.zip

--2020-02-15 14:51:32--  https://www.manythings.org/anki/spa-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 104.24.109.196, 2606:4700:3033::6818:6dc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4767708 (4.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   4.55M  17.3MB/s    in 0.3s    

2020-02-15 14:51:33 (17.3 MB/s) - ‘spa-eng.zip’ saved [4767708/4767708]

Archive:  spa-eng.zip
  inflating: _about.txt              
  inflating: spa.txt                 


In [2]:
!python -m spacy download en
!python -m spacy download es

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 11.1MB 17.9MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.1.0-cp36-none-any.whl size=11111556 sha256=312d43c959c0cd8fe37d426cdbf3e09fa79a39b982be8c9badf650e03a6a3b30
  Stored in directory: /tmp/pip-ephem-wheel-cache-f19z7qz_/wheels/cc/ee/c4/68922955901918a9aaa82e828d4f7ee1ccfc861285277e79b7
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/es_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/es
You can now load the model via spacy.load('es')


In [3]:
import torch

import torchtext
from torchtext.datasets import TranslationDataset
from torchtext import data

import spacy

from io import open

import unicodedata
import string
import re
import random
import os

print("Spacy version:", spacy.__version__)

Spacy version: 2.1.9


In [0]:
def unicodeToAscii(s):
    return ''.join(
        char for char in unicodedata.normalize('NFD', s)
        if unicodedata.category(char) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [0]:
def readPairs(pathToFile, slang, tlang):
    print("Reading lines...")

    f = open(pathToFile, encoding='utf-8')
    lines = f.read().strip().split('\n')

    pairs = [[normalizeString(s) for s in line.split('\t')[:2]] for line in lines]

    return pairs

In [0]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[0].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [0]:
def prepareData(pathToFile, slang, tlang):
    pairs = readPairs(pathToFile, slang, tlang)
    print("Read {} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {} sentence pairs".format(len(pairs)))
    return pairs

In [0]:
def datasetIndexes(pairs, train=0.8, val=0.1):
    num_examples = len(pairs)
    indexes = list(range(num_examples))
    last_train_idx = round(train*num_examples)
    last_valid_idx = last_train_idx + round(val*num_examples)
    random.shuffle(indexes)
    train_idxs = indexes[:last_train_idx]
    val_idxs = indexes[last_train_idx:last_valid_idx]
    test_idxs = indexes[last_valid_idx:]
    return train_idxs, val_idxs, test_idxs


In [0]:
class SpanishDataset(TranslationDataset):
    """English to Spanish Dataset"""

    @classmethod
    def splits(cls, exts, fields, root='.data/',
               train='train', validation='val', test='test', **kwargs):
        
        if 'path' not in kwargs:
            expected_folder = os.path.join(root, cls.name)
            path = expected_folder if os.path.exists(expected_folder) else None
        else:
            path = kwargs['path']
            del kwargs['path']
        
        return super(SpanishDataset, cls).splits(
            exts, fields, path, root, train, validation, test, **kwargs
        )

In [10]:
pairs = prepareData('spa.txt', 'eng', 'spa')
train_idxs, val_idxs, test_idxs = datasetIndexes(pairs)
print("{} {} {}".format(len(train_idxs), len(val_idxs), len(test_idxs) ))
print(random.choice(pairs))

Reading lines...
Read 123335 sentence pairs
Trimmed to 7588 sentence pairs
6070 759 759
['i am happy to see you again .', 'estoy feliz de verte otra vez .']


In [0]:
with open('train.en', 'w') as slang_file, open('train.es', 'w') as tlang_file:
    for i in train_idxs:
        slang_file.write(pairs[i][0] + '\n')
        tlang_file.write(pairs[i][1] + '\n')

with open('val.en', 'w') as slang_file, open('val.es', 'w') as tlang_file:
    for i in val_idxs:
        slang_file.write(pairs[i][0] + '\n')
        tlang_file.write(pairs[i][1] + '\n')

with open('test.en', 'w') as slang_file, open('test.es', 'w') as tlang_file:
    for i in test_idxs:
        slang_file.write(pairs[i][0] + '\n')
        tlang_file.write(pairs[i][1] + '\n')

In [0]:


spacy_es = spacy.load('es')
spacy_en = spacy.load('en')

def tokenize_es(text):
    """
    Tokenizes Spanish text from a string into a list of strings
    """
    return [token.text for token in spacy_es.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [token.text for token in spacy_en.tokenizer(text)]

In [13]:
!pwd
!ls -l

/content
total 23068
-rw-r--r-- 1 root root     1441 Jan 11 23:49 _about.txt
drwxr-xr-x 1 root root     4096 Feb  5 18:37 sample_data
-rw-r--r-- 1 root root  4767708 Jan 11 14:49 spa-eng.zip
-rw-r--r-- 1 root root 18432706 Jan 11 23:49 spa.txt
-rw-r--r-- 1 root root    19187 Feb 15 14:52 test.en
-rw-r--r-- 1 root root    20265 Feb 15 14:52 test.es
-rw-r--r-- 1 root root   155117 Feb 15 14:52 train.en
-rw-r--r-- 1 root root   164106 Feb 15 14:52 train.es
-rw-r--r-- 1 root root    19405 Feb 15 14:52 val.en
-rw-r--r-- 1 root root    20540 Feb 15 14:52 val.es


In [0]:
SRC = data.Field(tokenize=tokenize_en, init_token='<SOS>', eos_token='<EOS>', lower=True)
TRG = data.Field(tokenize=tokenize_es, init_token='<SOS>', eos_token='<EOS>', lower=True)

train_data, valid_data, test_data = SpanishDataset.splits(
    path='',
    exts=('.en', '.es'),
    fields=(SRC, TRG)
)

In [0]:
SRC.build_vocab(train_data, min_freq=5)
TRG.build_vocab(train_data, min_freq=5)

In [0]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device
)

In [17]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

6070
759
759


In [19]:
SRC.vocab.itos[:10]

['<unk>', '<pad>', '<SOS>', '<EOS>', '.', 'i', 'm', 're', 'you', 'he']

In [25]:
batch = next(iter(train_iterator))

src_example = batch.src[:, 0]
src = ' '.join(map(lambda i: SRC.vocab.itos[i], src_example))
trg_example = batch.trg[:, 0]
trg = ' '.join(map(lambda i: TRG.vocab.itos[i], trg_example))

print("English sentence:", src)
print("Spanish sentence:", trg)


English sentence: <SOS> i m taller than him . <EOS> <pad> <pad> <pad>
Spanish sentence: <SOS> soy mas alto que el . <EOS> <pad> <pad> <pad>


In [26]:
SRC.vocab.freqs.most_common(20)

[('.', 5988),
 ('i', 2793),
 ('m', 2192),
 ('re', 1433),
 ('you', 1378),
 ('he', 1084),
 ('is', 975),
 ('to', 920),
 ('a', 915),
 ('s', 651),
 ('not', 606),
 ('the', 548),
 ('we', 529),
 ('she', 521),
 ('am', 514),
 ('of', 411),
 ('are', 402),
 ('they', 380),
 ('very', 316),
 ('in', 314)]

In [27]:
TRG.vocab.freqs.most_common(20)

[('.', 5970),
 ('estoy', 1281),
 ('el', 1083),
 ('de', 945),
 ('es', 902),
 ('no', 786),
 ('a', 680),
 ('soy', 553),
 ('un', 547),
 ('esta', 534),
 ('que', 443),
 ('ella', 420),
 ('en', 401),
 ('la', 378),
 ('eres', 357),
 ('me', 340),
 ('muy', 317),
 ('estamos', 265),
 ('una', 256),
 ('voy', 237)]